In [53]:
from trading import api

trades_unit = api.fetch_data()
trades_date = api.convert_to_time(trades_unit)
trades_unit = api.set_balances(trades_unit)
trades_date = api.set_balances(trades_date.reset_index()).set_index('Date')

trades_unit

,Gain,Gain_Globally,Balance_Globally,Gain_Yearly_2021,Balance_Yearly_2021,Gain_Monthly_2021-8,Balance_Monthly_2021-8,Gain_Weekly_2021-34,Balance_Weekly_2021-34,Gain_Weekly_2021-35,...,Gain_Monthly_2023-3,Balance_Monthly_2023-3,Gain_Weekly_2023-10,Balance_Weekly_2023-10,Gain_Weekly_2023-11,Balance_Weekly_2023-11,Gain_Weekly_2023-12,Balance_Weekly_2023-12,Gain_Weekly_2023-13,Balance_Weekly_2023-13
Date,,,,,,,,,,,,,,,,,,,,,
2021-08-27,35.09,35.09,35.09,35.09,35.09,35.09,35.09,35.09,35.09,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-08-28,NaN,NaN,35.09,NaN,35.09,NaN,35.09,NaN,35.09,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-08-29,NaN,NaN,35.09,NaN,35.09,NaN,35.09,NaN,35.09,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-08-30,NaN,NaN,35.09,NaN,35.09,NaN,35.09,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-08-31,259.50,259.50,294.59,259.50,294.59,259.50,294.59,NaN,NaN,259.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-24,NaN,NaN,-166.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.00,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
2023-03-25,NaN,NaN,-166.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.00,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
2023-03-26,NaN,NaN,-166.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.00,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN


In [49]:
import streamlit as st
import gspread as gs 
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

gc = gs.service_account_from_dict(st.secrets['gcp_service_account'])
ss = gc.open_by_key(st.secrets['trading'].spreadsheet_key)
trades = pd.DataFrame(ss.worksheet('Trades').get_values('A:AD'))
trades = pd.DataFrame(trades.values[1:], columns=trades.iloc[0])
trades = trades.astype({'Date': 'datetime64[ns]', 'Gain': 'float64'})
trades = trades.sort_values('Date').reset_index(drop=True)[['Date', 'Gain']]

for idx in trades.index:
    trades.at[idx, 'Balance_Globally'] = np.sum(trades.loc[:idx]['Gain'])
    yearly = f"Yearly_{trades.at[idx, 'Date'].year}"
    trades.at[idx, f'Gain_{yearly}'] = np.sum(trades.at[idx, 'Gain'])
    trades.at[idx, f'Balance_{yearly}'] = np.sum(trades.loc[:idx][f'Gain_{yearly}'])
    monthly = f"Monthly_{trades.at[idx, 'Date'].year}-{trades.at[idx, 'Date'].month}"
    trades.at[idx, f'Gain_{monthly}'] = np.sum(trades.at[idx, 'Gain'])
    trades.at[idx, f'Balance_{monthly}'] = np.sum(trades.loc[:idx][f'Gain_{monthly}'])
    weekly = f"Weekly_{trades.at[idx, 'Date'].year}-{trades.at[idx, 'Date'].week}"
    trades.at[idx, f'Gain_{weekly}'] = np.sum(trades.at[idx, 'Gain'])
    trades.at[idx, f'Balance_{weekly}'] = np.sum(trades.loc[:idx][f'Gain_{weekly}'])

trades_time = pd.concat([
    pd.DataFrame({'Date': pd.date_range(start=trades['Date'].min(), end=trades['Date'].max())}).set_index('Date'),
    trades.set_index('Date').groupby('Date').agg({'Gain': 'sum'}),
], axis=1).reset_index()

for idx in trades_time.index:
    trades_time.at[idx, 'Balance_Globally'] = np.sum(trades_time.loc[:idx]['Gain'])
    yearly = f"Yearly_{trades_time.at[idx, 'Date'].year}"
    trades_time.at[idx, f'Gain_{yearly}'] = np.sum(trades_time.at[idx, 'Gain'])
    trades_time.at[idx, f'Balance_{yearly}'] = np.sum(trades_time.loc[:idx][f'Gain_{yearly}'])
    monthly = f"Monthly_{trades_time.at[idx, 'Date'].year}-{trades_time.at[idx, 'Date'].month}"
    trades_time.at[idx, f'Gain_{monthly}'] = np.sum(trades_time.at[idx, 'Gain'])
    trades_time.at[idx, f'Balance_{monthly}'] = np.sum(trades_time.loc[:idx][f'Gain_{monthly}'])
    weekly = f"Weekly_{trades_time.at[idx, 'Date'].year}-{trades_time.at[idx, 'Date'].week}"
    trades_time.at[idx, f'Gain_{weekly}'] = np.sum(trades_time.at[idx, 'Gain'])
    trades_time.at[idx, f'Balance_{weekly}'] = np.sum(trades_time.loc[:idx][f'Gain_{weekly}'])

trades_time = trades_time.set_index('Date')


fig = go.Figure()
fig.add_trace(go.Bar(x=trades.index.to_list(), y=trades['Gain'].to_list(), name='Trades'))
for col in trades.columns:
    if 'Balance' in col:
        fig.add_trace(go.Scatter(x=trades.index.to_list(), y=trades[col].to_list(), name=col))
fig.show()


fig = go.Figure()
fig.add_trace(go.Bar(x=trades_time.index.to_list(), y=trades_time['Gain'].to_list(), name='Trades'))
for col in trades_time.columns:
    if 'Balance' in col:
        fig.add_trace(go.Scatter(x=trades_time.index.to_list(), y=trades_time[col].to_list(), name=col))
fig.show()

trades_time[['Gain']].head(50)

,Gain
Date,
2021-08-27,35.09
2021-08-28,NaN
2021-08-29,NaN
2021-08-30,NaN
2021-08-31,259.50
2021-09-01,NaN
2021-09-02,4.32
2021-09-03,3.50
2021-09-04,NaN
